Tensor-Compressed PINN for Solving Burgers Equation

Backend/Functions

In [62]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.stats import qmc
import tensorlearn as tl
import keras
import tensorflow_datasets as tfds
import time

In [2]:
##nn basics

def read_network(network_path):

    saved_model = tf.keras.models.load_model(network_path)

    return saved_model



def layer_param(nn_model, layer_name):

    return nn_model.get_layer(layer_name).weights



def get_dense_layer_weights(param):
    return param[0].numpy()

def get_dense_layer_bias(param):
    return param[1].numpy()

def set_dense_layer_param_list(weights,bias):
    return [weights,bias]


    
def freez_layer(nn_model,index, param_layer):
    nn_model.layers[index].set_weights(param_layer)
    nn_model.layers[index].trainable=False
    return nn_model


def net_retrain(nn_model, ds_train, ds_test, epochs, output_path):


    checkpoint_path=output_path
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_accuracy',
                                                 save_weights_only=False,
                                                 save_best_only=True,
                                                 verbose=1)
    nn_model.fit(
                ds_train,
                epochs=epochs,
                validation_data=ds_test,
                callbacks=[cp_callback]
            )                


    

    nn_model.save(output_path)
    return nn_model


def get_layer_index_by_name(model, layer_name):
    for index, layer in enumerate(model.layers):
        if layer.name == layer_name:
            return index
        

def inference(nn_model, ds_test):
    return nn_model.evaluate(ds_test)



def mnist_data_load(batch_size=128):
        tfds.disable_progress_bar()
        #tf.enable_v2_behavior()
        
        (ds_train, ds_test), ds_info = tfds.load(
            'fashion_mnist',
            split=['train', 'test'],
            shuffle_files=True,
            as_supervised=True,
            with_info=True,
        )
        
        def normalize_img(image, label):
          """Normalizes images: `uint8` -> `float32`."""
          return tf.cast(image, tf.float32) / 255., label
        
        ds_train = ds_train.map(
            normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        ds_train = ds_train.cache()
        ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
        ds_train = ds_train.batch(batch_size)
        ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
        
        ds_test = ds_test.map(
            normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        ds_test = ds_test.batch(batch_size)
        ds_test = ds_test.cache()
        ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

        return ds_train, ds_test

In [291]:
#load model
#tcPINN = tf.keras.models.load_model("b_14x50.h5")
model_name = 'b_10x50.h5'
tcPINN = tf.keras.models.load_model(model_name)
tcPINN.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 2)]               0         
                                                                 
 dense_172 (Dense)           (None, 50)                150       
                                                                 
 dense_173 (Dense)           (None, 50)                2550      
                                                                 
 dense_174 (Dense)           (None, 50)                2550      
                                                                 
 dense_175 (Dense)           (None, 50)                2550      
                                                                 
 dense_176 (Dense)           (None, 50)                2550      
                                                                 
 dense_177 (Dense)           (None, 50)                255

In [4]:
#FIX THIS - GPU setup
#pip install tensorflow-gpu

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
    print("No GPU available.")
else:
    print("GPU(s) available:", len(physical_devices))

No GPU available.


In [292]:
#decomposing and recomposing a layer of weights
def decomp_recomp(weights, epsilon, shape, layer, save_layer):

    og_shape = np.array(weights.shape)
    #print(type(og_shape))
    np_weights = np.array(weights)
    #print(np_weights.shape)
    #print(type(np_weights))

    weights_3d = np.reshape(np_weights, shape).astype(np.float64) #shape search???
    #20x20 = 10 8 5
    #40x40 = 16 10 10
    #60x60 = 16 15 15

    #print(type(weights_3d))

    #decompose/recompose
    tt_weights = tl.auto_rank_tt(weights_3d, epsilon) #error of 0.5 - 1, tweak w shape search?
    recomp_weights = tl.tt_to_tensor(tt_weights)

    #data saving
    compression_ratio = tl.tt_compression_ratio(tt_weights)
    data_saving = 1 - (1/compression_ratio)
    print("data_saving (%)", data_saving*100)

    #save compressed layer
    if(save_layer):
        tt_weights_npy = np.array(tt_weights, dtype=object)
        np.save(f'{layer}.npy', tt_weights_npy)

    return np.reshape(recomp_weights, og_shape) #og shape

In [41]:
#calculate loss

### generating data

# number of boundary and initial data points
# value `Nd` in the reference paper:
# Nd = number_of_ic_points + number_of_bc1_points + number_of_bc1_points 
number_of_ic_points = 50
number_of_bc1_points = 25
number_of_bc2_points = 25

# Latin Hypercube Sampling (LHS) engine ; to sample random points in domain,
# boundary and initial boundary
engine = qmc.LatinHypercube(d=1)

# temporal data points
t_d = engine.random(n=number_of_bc1_points + number_of_bc2_points)
temp = np.zeros([number_of_ic_points, 1]) # for IC ; t = 0
t_d = np.append(temp, t_d, axis=0)
# spatial data points
x_d = engine.random(n=number_of_ic_points)
x_d = 2 * (x_d - 0.5)
temp1 = -1 * np.ones([number_of_bc1_points, 1]) # for BC1 ; x = -1
temp2 = +1 * np.ones([number_of_bc2_points, 1]) # for BC2 ; x = +1
x_d = np.append(x_d, temp1, axis=0)
x_d = np.append(x_d, temp2, axis=0)

# output values for data points (boundary and initial)
y_d = np.zeros(x_d.shape)

# for initial condition: IC = -sin(pi*x)
y_d[ : number_of_ic_points] = -np.sin(np.pi * x_d[:number_of_ic_points])

# all boundary conditions are set to zero
y_d[number_of_ic_points : number_of_bc1_points + number_of_ic_points] = 0
y_d[number_of_bc1_points + number_of_ic_points : number_of_bc1_points + number_of_ic_points + number_of_bc2_points] = 0

# number of collocation points
Nc = 10000

# LHS for collocation points
engine = qmc.LatinHypercube(d=2)
data = engine.random(n=Nc)
# set x values between -1. and +1.
data[:, 1] = 2*(data[:, 1]-0.5)

# change names
t_c = np.expand_dims(data[:, 0], axis=1)
x_c = np.expand_dims(data[:, 1], axis=1)

# MSE loss function
# IMPORTANT: this loss function is used for data points
@tf.function
def mse(y, y_):
    return tf.reduce_mean(tf.square(y-y_))

def calc_loss(model):

    # u(t, x) just makes working with model easier and the whole code looks more
    # like its mathematical backend
    @tf.function
    def u(t, x, model):
        # model input shape is (2,) and `u` recieves 2 arguments with shape (1,)
        # to be able to feed those 2 args (t, x) to the model, a shape (2,) matrix
        # is build by simply concatenation of (t, x)
        u = model(tf.concat([t, x], axis=1)) # note the axis ; `column`
        return u
    
    # the physics informed loss function
    # IMPORTANT: this loss function is used for collocation points
    @tf.function
    def f(t, x, model):
        u0 = u(t, x, model)
        u_t = tf.gradients(u0, t)[0]
        u_x = tf.gradients(u0, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        F = u_t + u0*u_x - (0.01/np.pi)*u_xx
        return tf.reduce_mean(tf.square(F))
    
    # model output/prediction
    y_ = u(t_d, x_d, model)
    # physics-informed loss for collocation points
    L1 = f(t_c, x_c, model)
    # MSE loss for data points
    L2 = mse(y_d, y_)
    loss = L1 + L2
    return loss.numpy()

In [42]:
#retrain model

def retrain(model, epochs):

    #loss funciton definitions
    # u(t, x) just makes working with model easier and the whole code looks more
    # like its mathematical backend
    @tf.function
    def u(t, x, model):
        # model input shape is (2,) and `u` recieves 2 arguments with shape (1,)
        # to be able to feed those 2 args (t, x) to the model, a shape (2,) matrix
        # is build by simply concatenation of (t, x)
        u = model(tf.concat([t, x], axis=1)) # note the axis ; `column`
        return u
    
    # the physics informed loss function
    # IMPORTANT: this loss function is used for collocation points
    @tf.function
    def f(t, x, model):
        u0 = u(t, x, model)
        u_t = tf.gradients(u0, t)[0]
        u_x = tf.gradients(u0, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        F = u_t + u0*u_x - (0.01/np.pi)*u_xx
        return tf.reduce_mean(tf.square(F))

    loss_list = []

    # L-BFGS optimizer was used in the reference paper
    opt = tf.keras.optimizers.Adam(learning_rate=5e-4)
    start = time.time()

    # training loop
    # IMPORTANT: a while-based training loop is more beneficial
    # updates the model while loss > 0.006
    for epoch in range(epochs + 1):
        with tf.GradientTape() as tape:
            # model output/prediction
            y_ = u(t_d, x_d, model)
            # physics-informed loss for collocation points
            L1 = f(t_c, x_c, model)
            # MSE loss for data points
            L2 = mse(y_d, y_)
            loss = L1 + L2
        # compute gradients
        g = tape.gradient(loss, model.trainable_weights)
        loss_list.append(loss)
        # log every 10 epochs
        if (not epoch%50) or (epoch == epochs):
            print(f"{epoch:4} {loss.numpy()}")
        # apply gradients
        opt.apply_gradients(zip(g, model.trainable_weights))

    end = time.time()
    print(f"{end - start:.10} (s)")

In [9]:
#reset tcPINN
og_model = tf.keras.models.load_model("burgers.h5") #loss = 0.00591
og_weights = og_model.get_weights()
tcPINN.set_weights(og_weights)
for i in range(11):
    tcPINN.layers[i].set_trainable = True


Actual TT Decomp and Retraining

In [289]:
#iterative TT decomp + retraining

#reset tcPINN
og_model = tf.keras.models.load_model(model_name)
og_weights = og_model.get_weights()
tcPINN.set_weights(og_weights)
for i in range(11): #num layers + 1
    tcPINN.layers[i].set_trainable = True

print(calc_loss(tcPINN))

# print('----------------- input layer -----------------')

# #get specific layer of weights
# layer = tcPINN.get_weights()[0]
# weights = tcPINN.get_weights()

# #compress layer
# compressed_layer = decomp_recomp(layer, 0.5, [6, 5, 4], 'input layer', True)

# #input compressed layer
# weights[0] = compressed_layer
# tcPINN.set_weights(weights)

# #freeze layer
# tcPINN.layers[0].trainable = False

# #retrain
# retrain(tcPINN, 1000)
for layer_index in range(1, 11): #num layers + 1

    print('-----------------', layer_index, '-----------------')

    #get specific layer of weights
    layer = tcPINN.get_weights()[layer_index * 2]
    weights = tcPINN.get_weights()

    #compress layer
    compressed_layer = decomp_recomp(layer, 1, [25, 10, 10], layer_index, False)
    #20x20 = [10, 8, 5]
    #30x30 = [10, 10, 9]
    #40x40 = [16, 10, 10]
    #50x50 = [25, 10, 10]

    #input compressed layer
    weights[layer_index * 2] = compressed_layer
    tcPINN.set_weights(weights)

    #freeze layer
    tcPINN.layers[layer_index].trainable = False

    #retrain
    retrain(tcPINN, 2000)

    print(tcPINN.get_weights()[layer_index * 2].shape)

#retrain(tcPINN, 4000)

    

    


0.0006366909955450453
----------------- 1 -----------------
data_saving (%) 90.44444444444444
   0 1.0963152636606994
  50 0.10531905384203098
 100 0.08503314799533111
 150 0.06542739105200296
 200 0.058611546209179785
 250 0.05519542842959165
 300 0.05256535884986857
 350 0.04941559975157655
 400 0.04501525788884743
 450 0.038074282861899775
 500 0.023970584036924
 550 0.03244921187475032
 600 0.014573194624540502
 650 0.00869245805753686
 700 0.04389853526486509
 750 0.004946856407866088
 800 0.0033078389079995805
 850 0.0026387887543837664
 900 0.0021917677154560835
 950 0.0018697226012918196
1000 0.0016255247938982737
1050 0.006132037276410684
1100 0.0013571752148198797
1150 0.0013950287567956873
1200 0.0015400470872849157
1250 0.004131961764590149
1300 0.0011152607910437382
1350 0.0019088377640982027
1400 0.0015356354075201236
1450 0.001322129776671813
1500 0.0007546567774496906
1550 0.0008346928769482655
1600 0.0007160113797080386
1650 0.0006696109437978608
1700 0.001985688794466

Reconstruction?

In [ ]:
#reconstruct 14x50 perchance

In [ ]:
#iterative TT decomp + retraining

#reset tcPINN
og_model = tf.keras.models.load_model(model_name)
og_weights = og_model.get_weights()
tcPINN.set_weights(og_weights)
for i in range(11): #num layers + 1
    tcPINN.layers[i].set_trainable = True

print(calc_loss(tcPINN))

# print('----------------- input layer -----------------')

# #get specific layer of weights
# layer = tcPINN.get_weights()[0]
# weights = tcPINN.get_weights()

# #compress layer
# compressed_layer = decomp_recomp(layer, 0.5, [6, 5, 4], 'input layer', True)

# #input compressed layer
# weights[0] = compressed_layer
# tcPINN.set_weights(weights)

# #freeze layer
# tcPINN.layers[0].trainable = False

# #retrain
# retrain(tcPINN, 1000)

for layer_index in range(1, 12): #num layers + 1

    print('-----------------', layer_index, '-----------------')

    #get specific layer of weights
    layer = tcPINN.get_weights()[layer_index * 2]
    weights = tcPINN.get_weights()

    #compress layer 
    #20x20 = [10, 8, 5]
    #30x30 = [10, 10, 9]
    #40x40 = 16 10 10
    #50x50 = 25 10 10

    #input compressed layer
    weights[layer_index * 2] = compressed_layer
    tcPINN.set_weights(weights)

    #freeze layer
    tcPINN.layers[layer_index].trainable = False

    #retrain
    retrain(tcPINN, 2000)

    print(tcPINN.get_weights()[layer_index * 2].shape)

#retrain(tcPINN, 4000)

In [290]:
tcPINN.save('tc_b_14x30.h5')

In [94]:
#compress input layer
print('----------------- input layer -----------------')

#get specific layer of weights
layer = tcPINN.get_weights()[0]
weights = tcPINN.get_weights()

#compress layer
compressed_layer = decomp_recomp(layer, 0.5, [6, 5, 4], 'input layer', False)

# #input compressed layer
# weights[0] = compressed_layer
# tcPINN.set_weights(weights)

# #freeze layer
# tcPINN.layers[0].trainable = False

# #retrain
# retrain(tcPINN, 1000)

----------------- input layer -----------------
data_saving (%) 19.999999999999996


In [68]:
#load compressed input layer

compressed_input_layer = np.load('input_layer_0.73_0.0069.npy')


In [268]:
tcPINN = tf.keras.models.load_model('b_8x40.h5')
tcPINN.summary()

for layer_index in range(1, 9):

    print('-----------------', layer_index, '-----------------')

    #get specific layer of weights
    layer = tcPINN.get_weights()[layer_index * 2]
    weights = tcPINN.get_weights()
    print(layer.shape)
    decomp_recomp(layer, 1, [16, 10, 10], layer_index, False)

Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 2)]               0         
                                                                 
 dense_330 (Dense)           (None, 40)                120       
                                                                 
 dense_331 (Dense)           (None, 40)                1640      
                                                                 
 dense_332 (Dense)           (None, 40)                1640      
                                                                 
 dense_333 (Dense)           (None, 40)                1640      
                                                                 
 dense_334 (Dense)           (None, 40)                1640      
                                                                 
 dense_335 (Dense)           (None, 40)                164

In [36]:
#save original weights
tcPINN = tf.keras.models.load_model("burgers.h5")

for layer_index in range(0, 10):

    #get specific layer of weights
    layer = tcPINN.get_weights()[layer_index * 2 + 1]
    np.save(f'{layer_index}.npy', layer)

In [287]:
model = tf.keras.models.load_model('b_14x30.h5')

print(calc_loss(model))
model.summary()
#retrain(model, 500)

0.0006366909955450453
Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 2)]               0         
                                                                 
 dense_284 (Dense)           (None, 30)                90        
                                                                 
 dense_285 (Dense)           (None, 30)                930       
                                                                 
 dense_286 (Dense)           (None, 30)                930       
                                                                 
 dense_287 (Dense)           (None, 30)                930       
                                                                 
 dense_288 (Dense)           (None, 30)                930       
                                                                 
 dense_289 (Dense)           (None, 